In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

In [2]:
import os
from pprint import pformat

import pytorch_lightning as pl
import torch
from omegaconf import OmegaConf
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.strategies.ddp import DDPStrategy

from mld.callback import ProgressLogger
from mld.config import parse_args
from mld.data.get_data import get_datasets
from mld.models.get_model import get_model
from mld.utils.logger import create_logger
import sys

In [7]:
sys.argv = ["train.py", "--cfg", "configs/config_vae_humanml3d.yaml", "--cfg_assets", "configs/assets.yaml", "--batch_size", "64", "--nodebug"]
cfg = parse_args()  # parse config file
logger = create_logger(cfg, phase="train")
datasets = get_datasets(cfg, logger=logger)
model = get_model(cfg, datasets[0])
data_loader = datasets[0].train_dataloader()

batch = next(iter(data_loader))
batch.keys()
batch['motion'].shape

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


cfg.TRAIN.BATCH_SIZE: 64
Pointer Pointing at 0


Output()

Pointer Pointing at 0


torch.Size([64, 196, 263])

In [7]:
model.allsplit_step(split='train', batch=batch, batch_idx=0)

tensor(0.7403, grad_fn=<AddBackward0>)

In [40]:
# from: mld/models/modeltype/mld.py: MLD:train_vae_forward
feats_ref = batch["motion"]
lengths = batch["length"]
motion_z, dist_m = model.vae.encode(feats_ref, lengths)
feats_rst = model.vae.decode(motion_z, lengths)
joints_rst = model.feats2joints(feats_rst)
joints_rst.shape

z: torch.Size([1, 64, 256])


torch.Size([64, 196, 22, 3])

In [36]:
motion_z.shape, dist_m.scale

(torch.Size([1, 64, 256]),
 tensor([[[0.7725, 1.0069, 0.4456,  ..., 0.5810, 2.2325, 1.2504],
          [0.7822, 1.2683, 0.7755,  ..., 0.4844, 2.7962, 1.2504],
          [0.5818, 1.8936, 0.8240,  ..., 0.9396, 2.7249, 0.9010],
          ...,
          [0.4743, 0.9068, 0.6290,  ..., 0.7586, 3.6105, 1.5674],
          [1.1121, 0.7113, 1.5482,  ..., 0.5164, 1.3498, 2.7461],
          [0.9881, 0.9536, 0.5386,  ..., 0.4572, 4.2650, 2.4904]]],
        grad_fn=<PowBackward0>))

In [34]:
model.is_vae

True

### Stage2

In [3]:
sys.argv = ["train.py", "--cfg", "configs/config_mld_humanml3d.yaml", "--cfg_assets", "configs/assets.yaml", "--batch_size", "64", "--nodebug"]
# python -m train --cfg configs/config_mld_humanml3d.yaml --cfg_assets configs/assets.yaml --batch_size 64 --nodebug
cfg = parse_args()  # parse config file
logger = create_logger(cfg, phase="train")
datasets = get_datasets(cfg, logger=logger)
model = get_model(cfg, datasets[0])
data_loader = datasets[0].train_dataloader()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Pointer Pointing at 0


Output()

Pointer Pointing at 0


In [4]:
batch = next(iter(data_loader))
batch.keys()
batch['motion'].shape


torch.Size([64, 196, 263])

In [5]:
model.allsplit_step(split='train', batch=batch, batch_idx=0)

noise_pred: torch.Size([64, 1, 256])


tensor(1.9939, grad_fn=<AddBackward0>)

### Demo

In [10]:
import logging
import os
import time
from builtins import ValueError
from multiprocessing.sharedctypes import Value
from pathlib import Path

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import ConcatDataset, DataLoader
# from torchsummary import summary
from tqdm import tqdm

from mld.config import parse_args
# from mld.datasets.get_dataset import get_datasets
from mld.data.get_data import get_datasets
from mld.data.sampling import subsample, upsample
from mld.models.get_model import get_model
from mld.utils.logger import create_logger
import sys

In [11]:
sys.argv = ["demo.py", "--cfg", "configs/config_mld_humanml3d.yaml", "--cfg_assets", "configs/assets.yaml", "--example", "./demo/example.txt"]
cfg = parse_args(phase="demo")
cfg.FOLDER = cfg.TEST.FOLDER
cfg.Name = "demo--" + cfg.NAME
logger = create_logger(cfg, phase="demo")
from mld.utils.demo_utils import load_example_input

text, length = load_example_input(cfg.DEMO.EXAMPLE)
task = "Example"

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(
            str(x) for x in cfg.DEVICE)
device = torch.device("cuda")

dataset = get_datasets(cfg, logger=logger, phase="test")[0]

# create mld model
total_time = time.time()
model = get_model(cfg, dataset)

state_dict = torch.load(cfg.TEST.CHECKPOINTS,
                            map_location="cpu")["state_dict"]

model.load_state_dict(state_dict, strict=True)

model.sample_mean = cfg.TEST.MEAN
model.fact = cfg.TEST.FACT
model.to(device)
model.eval()
''


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Length-50: a man kicks with something or someone with his left leg.
Length-100: A person is skipping rope.
Length-100: a person walks backward slowly.
Pointer Pointing at 0


''

In [ ]:
mld_time = time.time()
with torch.no_grad():
    rep_lst = []    
    rep_ref_lst = []
    texts_lst = []
    # task: input or Example
    if text:
        # prepare batch data
        batch = {"length": length, "text": text}
        
        for rep in range(cfg.DEMO.REPLICATION):
            # text motion transfer
            if cfg.DEMO.MOTION_TRANSFER:
                joints = model.forward_motion_style_transfer(batch)
            # text to motion synthesis
            else:
                joints = model(batch)

            # cal inference time
            infer_time = time.time() - mld_time
            num_batch = 1
            num_all_frame = sum(batch["length"])
            num_ave_frame = sum(batch["length"]) / len(batch["length"])

            # upscaling to compare with other methods
            # joints = upsample(joints, cfg.DATASET.KIT.FRAME_RATE, cfg.DEMO.FRAME_RATE)
            nsample = len(joints)
            id = 0
            for i in range(nsample):
                joints[i].detach().cpu().numpy()
            if cfg.DEMO.OUTALL:
                rep_lst.append(joints)
                texts_lst.append(batch["text"])
total_time = time.time() - total_time
print(
    f'Total time spent: {total_time:.2f} seconds (including model loading time and exporting time).'
)

In [25]:
# 2 models:
model.denoiser
model.vae.decoder
''

''

### DELETE ME

In [24]:
model.denoiser.ablation_skip_connection
# model.denoiser.query_pos.pe.shape
model.vae.arch
model.cfg.FOLDER_EXP

'results/mld/1222_PELearn_Diff_Latent1_MEncDec49_MdiffEnc49_bs64_clip_uncond75_01'

In [80]:
encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model=256,
            nhead=4,
            dim_feedforward=1024,
            activation='gelu')
encoder = torch.nn.TransformerEncoder(encoder_layer,
                                        num_layers=9)

In [126]:
data = torch.rand(3, 6, 256)
_ = encoder(data)